### Import thư viện

In [1]:
import os
import re
import json

In [2]:
import pandas as pd

In [3]:
import spacy
# Tải mô hình ngôn ngữ tiếng Anh từ spaCy
nlp = spacy.load("en_core_web_lg")

---

### Đọc và tiền xử lý dữ liệu cho quá trình train mô hình

In [4]:
path_folder = "../data_summarize/"
files = os.listdir(path_folder)

In [5]:
data_list = []
for file in files:
    with open(f"{path_folder}{file}", "r", encoding="utf-8") as f:
        data_list.append(json.load(f))

In [6]:
df = pd.DataFrame(data_list, index=range(len(data_list)))

In [7]:
df

,article,highlights
0,By . Bianca London . She's no stranger to bein...,Coronation Street star tops poll with 39 per c...
1,"By . James Slack, Home Affairs Editor . PUBLIS...",New proposals will mean deportees can only app...
2,By . Lizzie Parry . John and Corinne Clarkson ...,John and Corinne Clarkson cooked up the 5ft 4i...
3,"December 19, 2014 . CNN Student News is wrappi...",This page includes the show Transcript .Use th...
4,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
...,...,...
99995,"(CNN) -- ""I learned that courage was not the a...",Kevin-Prince Boateng's walk-off protest marked...
99996,By . Daily Mail Reporter . PUBLISHED: . 09:33 ...,The teacher at Hyatt Elementary School in Lind...
99997,Spurs still so far from the top four . It was ...,Tottenham were thrashed by Manchester City onc...
99998,By . Daily Mail Reporter . PUBLISHED: . 13:19 ...,"Speedway coach Michael Green, 71, jailed for n..."


In [5]:
# Biểu thức chính quy để xác định các ký tự đặc biệt nằm giữa hai số
pattern = r'(?<!\d)[^\w\s%](?!\d)'

def preprocessing(sentence, type=None):

    # loại bỏ một số thẻ HTML nếu còn tồn tại
    clean_text = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence, flags=re.MULTILINE)
    clean_text= re.sub(r'\<a href', ' ', clean_text)
    clean_text = re.sub(r'&amp;', '', clean_text) 
    clean_text = re.sub(r'<br />', ' ', clean_text)
    clean_text = re.sub(r'\\"', ' ', clean_text)

    doc = nlp(sentence)
    lemmatized_text = " ".join([token.lemma_ for token in doc if not token.is_stop]).lower().strip()

    # Xóa các ký tự đặc biệt không thuộc trường hợp đã nêu
    lemmatized_text = re.sub(fr'(?<!\d)[^a-zA-Z0-9\s]|[^a-zA-Z0-9\s%](?!\d)|{pattern}', '', lemmatized_text)
    lemmatized_text = re.sub(r'\s+', ' ', lemmatized_text)

    if type == "summary":
        return "<sostok> " + lemmatized_text + " <eostok>"
    return lemmatized_text

In [11]:
df['article'] = df['article'].apply(preprocessing)
df['highlights'] = df['highlights'].apply(preprocessing, type="summary")

In [8]:
df.to_csv("../../data.csv", index=False)

---

Quá trình train mô hình Tóm Tắt được mô tả kỹ ở hai link dưới đây:
- https://www.kaggle.com/code/vannguyenthi/text-summarization-with-seq2seq-model
- 